# 文本分类实战

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [5]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [14]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]['review'], self.data.iloc[index]['label']

    def __len__(self):
        return len(self.data)

In [16]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [17]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [18]:
for i in range(10):
    print(trainset[i])

('交通便利,位于市中心;设置齐备,有车库.不足处是临街隔音效果太差,建议用双层窗户,其次为过道隔音不好,建议能都铺上厚点的地毯．总体而言，这家酒店性价比还可以．', 1)
('酒店不错,离我要公干的地方很近.房间设施很好,服务也很好,物有所值.', 1)
('早餐还可以，停车方便，二楼餐厅很安静。朝南的房间靠近锅炉房，声音吵。宾馆反馈2008年8月12日：回复：尊敬的宾客：首先非常感谢您对本酒店的肯定。关于您反映的房间外噪音问题，是酒店一项临时性的施工引起，我们对施工的噪音已有具体的要求和措施，也将更加严格执行。诚挚欢迎您的再次光临！', 1)
('这次定的是B区的标间，浴室比A区的标间多了浴缸，而且稍大，但房间设施虽然干净但毕竟只能够得上3星。好在早餐不错，可以加分，外加室外的环境真的不错，早起散步感觉很好。在安吉据说这里仍是最好的宾馆，看看每天那么多团体住客就能说明问题了。', 1)
('周边环境不好，虽然在五环边上，但交通也不十分便捷，初次去，恐怕不太好找。不过酒店本身还不错，挺新的，停车也很方便。由于只住了一晚，就匆匆离开了，对服务什么的，没太深切的感受。总体感觉还不错。', 1)
('靠近路边的房间太吵太吵了，根本没有办法睡觉。劝大家千万不要去了。', 0)
('这家酒店是我帮巴西客人订的，据客人说还不错，而且到机场也很方便。', 1)
('房间太小。设施太简单。服务太冷淡。不会再住。', 0)
('酒店太旧了，价格偏高，所谓送的三餐没有什么优势可言，总要出门办事，不可能天天跑回酒店去吃中餐和晚餐', 1)
('和LP一起订的度假，发现三合比较紧俏，于是赶紧下了订单。在三合感觉中国人绝对是“少数民族”估计老外对这个酒店可能比较熟悉，早餐是在酒店外（第一次这种体会，呵呵），出门向左50米（路对面），叫阳光和酒吧。只有第二天早上看到一对中国人，其他全是老外，感觉有点奇怪。早餐内容太少了。酒店内比较别致，很有古城的味道，房间内的家具比较紧凑，不过两个人住应该还好了。热水似乎不行，放了很长时间都是温水。电视奇高无比，晚上看会电视脖子酸，信号也一般。不过如果是体验古城味道，的确是个不错的选择。', 1)


## Step5 创建Dataloader

In [22]:
import torch

tokenizer = AutoTokenizer.from_pretrained('../../models/rbt3/')

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=12, padding='max_length', truncation=True,return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs



In [23]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [29]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[  101,  3146,   860,   677,  2697,  6230,  6820,   679,  7231,  8024,
           678,   102],
        [  101,  2791,  7313,  1377,   809,   117,  4294,  1166,  4638,  1947,
          2791,   102],
        [  101,  6858,  6814,  3025,  4923,  7564,  2137,  4638,  6983,  2421,
           117,   102],
        [  101,  6983,  2421,  4696,  4638,   679,  2582,   720,  3416,  8024,
           679,   102],
        [  101,  6983,  2421,  4638,   855,  5390,  2523,  1962,   117,  2218,
          1762,   102],
        [  101,  2600,  1378,  3302,  1218,  1447,   794,  2769,  3341,  1168,
          6624,   102],
        [  101,   769,  6858,  6820,  1377,   809,  8024,  2802,  6756,  4638,
          6413,   102],
        [  101,  1157,  1343,  4638,  3198,   952,  3300,  4157,  7410,  2823,
           117,   102],
        [  101,  3680,  3613,  1168,  2382,  2336,  6963,  3221,   857,  6821,
           702,   102],
        [  101,  6983,  2421,  2899,  4277,   123,  3215,  

## Step6 创建模型及优化器

In [30]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained('../../models/rbt3/')
if torch.cuda.is_available():
    model = model.cuda()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-10-13 11:59:29.351314: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-13 11:59:29.388407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 11:59:30.184005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../models/rbt3/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be ab

In [31]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [34]:
def evaluate():
    model.eval()
    acc_num = 0

    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k, v in batch.items()}

            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch['labels'].long()).float().sum()

    return acc_num / len(validset)


def train(epoch=10, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()

            if global_step % log_step == 0:
                print(f'ep : {ep}, global_step: {global_step}, loss: {output.loss.item()}')
            global_step += 1

        acc = evaluate()
        print(f'ep: {ep}, acc: {acc}')

## Step8 模型训练

In [35]:
train()

ep : 0, global_step: 0, loss: 0.3042250871658325
ep : 0, global_step: 100, loss: 0.3266545534133911
ep : 0, global_step: 200, loss: 0.3254771828651428
ep: 0, acc: 0.7886597514152527
ep : 1, global_step: 300, loss: 0.3195094168186188
ep : 1, global_step: 400, loss: 0.25848037004470825
ep: 1, acc: 0.7847937941551208
ep : 2, global_step: 500, loss: 0.30616986751556396
ep : 2, global_step: 600, loss: 0.11087875068187714
ep: 2, acc: 0.7706185579299927
ep : 3, global_step: 700, loss: 0.20774158835411072
ep : 3, global_step: 800, loss: 0.06977453827857971
ep: 3, acc: 0.7654638886451721
ep : 4, global_step: 900, loss: 0.45530325174331665
ep : 4, global_step: 1000, loss: 0.0937582403421402
ep: 4, acc: 0.7603092789649963
ep : 5, global_step: 1100, loss: 0.02936185896396637
ep : 5, global_step: 1200, loss: 0.020834796130657196
ep : 5, global_step: 1300, loss: 0.08647172152996063
ep: 5, acc: 0.7719072103500366
ep : 6, global_step: 1400, loss: 0.03320871293544769
ep : 6, global_step: 1500, loss: 0.

## Step9 模型预测

In [38]:
sen = '我觉得这家酒店不错，饭很好吃！'
id2label = {0: '差评！', 1:'好评！'}
model.eval()

with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors='pt')
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f'输入: {sen}\n模型预测结果: {id2label.get(pred.item())}')

输入: 我觉得这家酒店不错，饭很好吃！
模型预测结果: 好评！


In [39]:
from transformers import pipeline

model.config.id2label = id2label
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0)

In [40]:
pipe(sen)

[{'label': '好评！', 'score': 0.9999101161956787}]